In [1]:
# Code forked from Sweta Agrawal et al. paper 2018

import sys
# Changed to add python 3 BHG
sys.path.append('/home/bruce/anaconda3/envs/conda_clone/bin/python')
# adjust warnings as needed
import warnings
warnings.filterwarnings('ignore')  # "error", "ignore", "always", "default", "module" or "once"

import numpy as np
import os, sys, getopt, pickle, csv, sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix, make_scorer, recall_score, precision_score, classification_report, precision_recall_fscore_support
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble  import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
from textblob import TextBlob
import random
import matplotlib.pyplot as plt
from sklearn import metrics
from collections import Counter
import argparse
from sklearn.model_selection import cross_validate
from sklearn.metrics import roc_auc_score    
import preprocessor as p

In [2]:
# Set of Traditional Models using SVM, Naive Bayes, Linear Regression, and Random Forest

models = [ 'svm', 'naive', 'lr', 'random_forest']
NO_OF_FOLDS = 10
MODEL_TYPE = "all"
HASH_REMOVE = None

In [3]:
def load_data(filename):
    data = pickle.load(open(filename, 'rb'))
    x_text = []
    labels = []
    for i in range(len(data)):
        if(HASH_REMOVE):
            x_text.append(p.tokenize((data[i]['text']).encode('utf-8')))
        else:
            x_text.append(data[i]['text'])
        labels.append(data[i]['label'])
    return x_text,labels

def get_filename(dataset):
    global N_CLASS, HASH_REMOVE
    if(dataset=="twitter"):
        filename = "data/twitter_data.pkl"
        N_CLASS = 3
        HASH_REMOVE = False
    elif(dataset=="formspring"):
        N_CLASS = 2
        filename = "data/formspring_data.pkl"
        HASH_REMOVE = False
    elif(dataset=="wiki"):
        N_CLASS = 2
        filename = "data/wiki_data.pkl"
        HASH_REMOVE = False
    return filename

In [4]:
def get_scores(y_true, y_pred):
#     if(data=="wiki"):
#         auc = roc_auc_score(y_true,y_pred)
#         print('Test ROC AUC: %.3f' %auc)
#     print(":: Confusion Matrix")
#     print(confusion_matrix(y_true, y_pred))
#     print(":: Classification Report")
#     print(classification_report(y_true, y_pred))
    return np.array([ 
            precision_score(y_true, y_pred, average=None), 
            recall_score(y_true, y_pred,  average=None),
            f1_score(y_true, y_pred, average=None)])
    
def print_scores(scores):
    for i in range(N_CLASS):
        if(i!=0):
            # Changed print syntax from py2 to py3 by adding parentheses BHG
            print ("Precision Class %d (avg): %0.3f (+/- %0.3f)" % (i,scores[:, i].mean(), scores[:, i].std() * 2))
            print ("Recall Class %d (avg): %0.3f (+/- %0.3f)" % (i,scores[:,  N_CLASS+i].mean(), scores[:,N_CLASS+i].std() * 2))
            print ("F1_score Class %d (avg): %0.3f (+/- %0.3f)" % (i,scores[:, N_CLASS*2+i].mean(), scores[:,  N_CLASS*2+i].std() * 2))


In [5]:
def classification_model(X, Y, model_type):
    X, Y = shuffle(X, Y, random_state=42)
    # Changed print syntax from py2 to py3 by adding parentheses BHG
    print ("Model Type:", model_type)
    kf = KFold(n_splits=NO_OF_FOLDS)
    scores = []
    for train_index, test_index in kf.split(X):
        Y = np.asarray(Y)
        model = get_model(model_type)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        model.fit(X_train,y_train)
        y_pred = model.predict(X_test)
        curr_scores = get_scores(y_test, y_pred)
        scores.append(np.hstack(curr_scores))
    print_scores(np.array(scores))

In [6]:
def get_model(m_type):
    if m_type == 'lr':
        # Logistic Regression (aka logit, MaxEnt) classifier.
        logreg = LogisticRegression(class_weight="balanced")
    elif m_type == 'naive':
        # Naive Bayes classifier for multinomial models
        logreg =  MultinomialNB()
    elif m_type == "random_forest":
        # A random forest classifier
        logreg = RandomForestClassifier(n_estimators=100, n_jobs=-1)
    elif m_type == "svm":
        # Linear Support Vector Classification
        logreg = LinearSVC(class_weight="balanced")
    else:
        print ("ERROR: Please specify a correst model")
        return None
    return logreg

In [7]:
def train(x_text, labels, MODEL_TYPE):
    
    if(WORD):
        print("Using word based features")
        bow_transformer = CountVectorizer(analyzer="word",max_features = 10000,stop_words='english').fit(x_text)
        comments_bow = bow_transformer.transform(x_text)
        tfidf_transformer = TfidfTransformer(norm = 'l2').fit(comments_bow)
        comments_tfidf = tfidf_transformer.transform(comments_bow)
        features = comments_tfidf
    else: 
        print("Using char n-grams based features")
        bow_transformer = CountVectorizer(max_features = 10000, ngram_range = (1,2)).fit(x_text)
        comments_bow = bow_transformer.transform(x_text)
        tfidf_transformer = TfidfTransformer(norm = 'l2').fit(comments_bow)
        comments_tfidf = tfidf_transformer.transform(comments_bow)
        features = comments_tfidf
    
    if(data == "twitter"):
        dict1 = {'racism':0,'sexism':1,'none':2}
        labels = np.array([dict1[b] for b in labels])
    
    from collections import Counter
    print(Counter(labels))
    
    if(MODEL_TYPE != "all"):
        classification_model(features, labels, MODEL_TYPE)
    else:
        for model_type in models:
            classification_model(features, labels, model_type)

In [32]:
# Experiment 1a - Formspring data set using Bag of Words
data = "formspring"
WORD =  False

# Load the data set
x_text, labels = load_data(get_filename(data)) 
print ("Data loaded!\n")

# Examine the data
print("Raw Data\n\n",x_text[0],"\n")

#Examine the labels
print("First 100 Labels - 0 = neutral, 1 = bullying\n\n",np.array(labels[0:99]).transpose())

Data loaded!

Raw Data

 what is your favorite song ? d i like too many songs to have a favorite 

First 100 Labels - 0 = neutral, 1 = bullying

 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [27]:
# Run Experiment 1a - ['Formspring','BOW'] - currently set to all models

train(x_text, labels, MODEL_TYPE)

Data loaded!
Using char n-grams based features
Counter({0: 11997, 1: 776})
Model Type: svm
Precision Class 1 (avg): 0.466 (+/- 0.109)
Recall Class 1 (avg): 0.503 (+/- 0.122)
F1_score Class 1 (avg): 0.483 (+/- 0.104)
Model Type: naive
Precision Class 1 (avg): 0.850 (+/- 0.640)
Recall Class 1 (avg): 0.015 (+/- 0.015)
F1_score Class 1 (avg): 0.030 (+/- 0.028)
Model Type: lr
Precision Class 1 (avg): 0.410 (+/- 0.099)
Recall Class 1 (avg): 0.626 (+/- 0.131)
F1_score Class 1 (avg): 0.495 (+/- 0.104)
Model Type: random_forest
Precision Class 1 (avg): 0.782 (+/- 0.194)
Recall Class 1 (avg): 0.171 (+/- 0.067)
F1_score Class 1 (avg): 0.279 (+/- 0.092)


In [34]:
# Experiment 1b - Formspring data set using n-grams
data = "formspring"
WORD = True

x_text, labels = load_data(get_filename(data)) 
print ("Data loaded!")

# Examine the data
print("Raw Data\n\n",x_text[0],"\n")

#Examine the labels
print("Labels 0 = neutral, 1 = bullying\n\n",np.array(labels[0:100]).transpose())

Data loaded!
Raw Data

 what is your favorite song ? d i like too many songs to have a favorite 

Labels 0 = neutral, 1 = bullying

 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [35]:
# Run Experiment 1b - ['Formspring','n-grams'] - currently set to all models

train(x_text, labels, MODEL_TYPE)

Using word based features
Counter({0: 11997, 1: 776})
Model Type: svm
Precision Class 1 (avg): 0.415 (+/- 0.089)
Recall Class 1 (avg): 0.525 (+/- 0.132)
F1_score Class 1 (avg): 0.463 (+/- 0.100)
Model Type: naive
Precision Class 1 (avg): 0.575 (+/- 0.950)
Recall Class 1 (avg): 0.013 (+/- 0.029)
F1_score Class 1 (avg): 0.025 (+/- 0.055)
Model Type: lr
Precision Class 1 (avg): 0.407 (+/- 0.079)
Recall Class 1 (avg): 0.617 (+/- 0.127)
F1_score Class 1 (avg): 0.489 (+/- 0.084)
Model Type: random_forest
Precision Class 1 (avg): 0.679 (+/- 0.226)
Recall Class 1 (avg): 0.163 (+/- 0.066)
F1_score Class 1 (avg): 0.262 (+/- 0.099)


In [40]:
data = "twitter"
WORD = False
x_text, labels = load_data(get_filename(data)) 
print ("Data loaded!")

#Examine the labels
print("Labels ['sexism','racism','none']\n\n",np.array(labels[0:9]).transpose(),"\n")

# Examine the data
print("None example\n\n",x_text[0],"\n")
print("Racism example\n\n", x_text[6],"\n")
print("Sexism example\n\n",x_text[8])

Data loaded!
Labels ['sexism','racism','none']

 ['none' 'none' 'none' 'none' 'none' 'none' 'racism' 'racism' 'sexism'] 

None example

 rt @colonelkickhead: another bloody instant restaurant week ?  !  ?  !  seriously !  they just jumped the shark riding two other sharks powered by sh… 

Racism example

 @dianh4 @exposefalsehood and it is muslims who were the first crusaders ,  attacking the christian world for centuries before it attacked back 

Sexism example

 rt @fruitondabottom: #feminismisequalitywhen men are actually listened to and part of the dialog. #heforshe #womenagainstfeminism http://t.…


In [ ]:
train(x_text, labels, MODEL_TYPE)

In [11]:
data = "twitter"
WORD = True
x_text, labels = load_data(get_filename(data)) 
print ("Data loaded!")
train(x_text, labels, MODEL_TYPE)

Data loaded!
Using word based features
Counter({2: 11036, 1: 3117, 0: 1937})
Model Type: svm
Precision Class 1 (avg): 0.803 (+/- 0.044)
Recall Class 1 (avg): 0.744 (+/- 0.052)
F1_score Class 1 (avg): 0.772 (+/- 0.037)
Precision Class 2 (avg): 0.893 (+/- 0.023)
Recall Class 2 (avg): 0.901 (+/- 0.018)
F1_score Class 2 (avg): 0.897 (+/- 0.009)
Model Type: naive
Precision Class 1 (avg): 0.904 (+/- 0.035)
Recall Class 1 (avg): 0.469 (+/- 0.056)
F1_score Class 1 (avg): 0.617 (+/- 0.051)
Precision Class 2 (avg): 0.806 (+/- 0.022)
Recall Class 2 (avg): 0.963 (+/- 0.007)
F1_score Class 2 (avg): 0.877 (+/- 0.011)
Model Type: lr
Precision Class 1 (avg): 0.832 (+/- 0.039)
Recall Class 1 (avg): 0.663 (+/- 0.083)
F1_score Class 1 (avg): 0.738 (+/- 0.062)
Precision Class 2 (avg): 0.875 (+/- 0.026)
Recall Class 2 (avg): 0.916 (+/- 0.012)
F1_score Class 2 (avg): 0.895 (+/- 0.012)
Model Type: random_forest
Precision Class 1 (avg): 0.873 (+/- 0.044)
Recall Class 1 (avg): 0.639 (+/- 0.068)
F1_score Class 

In [12]:
data = "wiki"
WORD = False
x_text, labels = load_data(get_filename(data)) 
print ("Data loaded!")
train(x_text, labels, MODEL_TYPE)

Data loaded!
Using char n-grams based features
Counter({0: 102274, 1: 13590})
Model Type: svm
Precision Class 1 (avg): 0.591 (+/- 0.025)
Recall Class 1 (avg): 0.823 (+/- 0.020)
F1_score Class 1 (avg): 0.688 (+/- 0.017)
Model Type: naive
Precision Class 1 (avg): 0.839 (+/- 0.010)
Recall Class 1 (avg): 0.554 (+/- 0.028)
F1_score Class 1 (avg): 0.667 (+/- 0.021)
Model Type: lr
Precision Class 1 (avg): 0.602 (+/- 0.023)
Recall Class 1 (avg): 0.845 (+/- 0.022)
F1_score Class 1 (avg): 0.703 (+/- 0.017)
Model Type: random_forest
Precision Class 1 (avg): 0.887 (+/- 0.008)
Recall Class 1 (avg): 0.547 (+/- 0.033)
F1_score Class 1 (avg): 0.676 (+/- 0.025)


In [13]:
data = "wiki"
WORD = True
x_text, labels = load_data(get_filename(data)) 
print ("Data loaded!")
train(x_text, labels, MODEL_TYPE)

Data loaded!
Using word based features
Counter({0: 102274, 1: 13590})
Model Type: svm
Precision Class 1 (avg): 0.590 (+/- 0.022)
Recall Class 1 (avg): 0.818 (+/- 0.026)
F1_score Class 1 (avg): 0.686 (+/- 0.019)
Model Type: naive
Precision Class 1 (avg): 0.899 (+/- 0.016)
Recall Class 1 (avg): 0.522 (+/- 0.036)
F1_score Class 1 (avg): 0.660 (+/- 0.027)
Model Type: lr
Precision Class 1 (avg): 0.620 (+/- 0.027)
Recall Class 1 (avg): 0.834 (+/- 0.023)
F1_score Class 1 (avg): 0.711 (+/- 0.021)
Model Type: random_forest
Precision Class 1 (avg): 0.812 (+/- 0.022)
Recall Class 1 (avg): 0.663 (+/- 0.027)
F1_score Class 1 (avg): 0.730 (+/- 0.021)
